In [ ]:
import matplotlib.pyplot as plt
import scipy
import numpy as np
import time
import util
from keras.models import Model
from keras.layers import Dense
from keras.layers import LSTM, Input

import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split


2022-10-04 15:06:25.941615: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-04 15:06:30.035279: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-04 15:06:42.681309: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /dcsrsoft/spack/arolle/v1.0/spack/opt/spack/linux-rhel8-zen2/gcc-10.4.0/miniconda3-4.10.3-gpvric5au5ue2cp2qiiar6vijzx4ibnb/lib:/dcsrsoft/spack/arolle/v1.0/spack/opt/spack/linux-rhel8-zen/gcc-8.4.1/cudnn-8.4.0.27-

In [ ]:

import os

precision = 0.005  # m/z precision for raw data
data_per_sec = 2  # spectrum per second
half_time_window = 30  # time range before and after RT in sec
number_ticks = half_time_window * 2 * data_per_sec  # number of spectrum per data
intensity_treshold = 8000 #  everything under this will be annotated false
number_sample = 120


data_folder = '../data/sara_training/'

try:
    os.mkdir(data_folder)
except:
    pass



peak_clarkii1= data_folder + 'clarkii_peaks.csv' # data with peak: mz, rt, Y/N
mz_clarkii1 = data_folder + 'clarkii_T0I1.mzML'

peak_clarkii10= data_folder + 'AclarkiiT0I10ctrl_peaks.csv' # data with peak: mz, rt, Y/N
mz_clarkii10 = data_folder + 'AclarkiiT0I10ctrl.mzML'

peak_viridis= data_folder + 'viridis_BEHamide_peaks.csv' # data with peak: mz, rt, Y/N
mz_viridis = data_folder + 'viridisT0I2.mzML'

pickle_path = './pickle_training.npy'

In [ ]:
save_np = False
if save_np == True:

    data_clark1 = util.build_data_ml(peak_clarkii1,mz_clarkii1 )
    data_clark10 = util.build_data_ml(peak_clarkii10,mz_clarkii10, mz_rt_sec=False)
    data_viri = util.build_data_ml(peak_viridis,mz_viridis )

    all_data = [None,None,None,None]
    all_data[0] = data_clark1[0] + data_clark10[0] + data_viri[0]
    all_data[1] = data_clark1[1] + data_clark10[1] +  data_viri[1]
    all_data[2] = data_clark1[2] + data_clark10[2] + data_viri[2]
    all_data[3] = data_clark1[3] + data_clark10[3] + data_viri[3]

    with open(pickle_path, 'wb') as fi:
        np.save(fi, np.asanyarray(data_clark10, dtype=object))
    data = np.load(pickle_path, allow_pickle=True)

else:
    data = np.load(pickle_path, allow_pickle=True)
 

In [ ]:


X_ = data[0]
y_ =  data[1]
mz_ = data[2]
rt_ =  data[3]

import random
for i in random.sample(range(0,len(X_)),20):
    color = 'green' if y_[i] == 1 else 'red'
    yn = 'YES' if y_[i] == 1 else 'NO'
    plt.title(  str(i) + ' ' + yn + ' - ' + str(mz_[i]) + ' // ' + str(float(rt_[i])))
    plt.plot(X_[i], color=color)
    plt.show()


In [ ]:

def smooth(x,window_len=11,window='hanning'):
    """smooth the data using a window with requested size.

    This method is based on the convolution of a scaled window with the signal.
    The signal is prepared by introducing reflected copies of the signal
    (with the window size) in both ends so that transient parts are minimized
    in the begining and end part of the output signal.

    input:
        x: the input signal
        window_len: the dimension of the smoothing window; should be an odd integer
        window: the type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
            flat window will produce a moving average smoothing.

    output:
        the smoothed signal

    example:

    t=linspace(-2,2,0.1)
    x=sin(t)+randn(len(t))*0.1
    y=smooth(x)

    see also:

    numpy.hanning, numpy.hamming, numpy.bartlett, numpy.blackman, numpy.convolve
    scipy.signal.lfilter

    TODO: the window parameter could be the window itself if an array instead of a string
    NOTE: length(output) != length(input), to correct this: return y[(window_len/2-1):-(window_len/2)] instead of just y.
    """


    if window_len<3:
        return x


    s=np.r_[x[window_len-1:0:-1],x,x[-2:-window_len-1:-1]]
    #print(len(s))
    if window == 'flat': #moving average
        w=np.ones(window_len,'d')
    else:
        w=eval('np.'+window+'(window_len)')

    y=np.convolve(w/w.sum(),s,mode='valid')
    return y


def NormalizeData(data):
    if np.max(data) - np.min(data) == 0.0:
        return data
    return (data - np.min(data)) / (np.max(data) - np.min(data))


def getStat(data):
    s = [0, 0, 0, 0, 0]  # nbrObs, mean, var,skew, kurtosis

    if len(data) > 0:
        de = scipy.stats.describe(data)
        s = [de[0]/number_sample, de[2], de[3], de[4], de[5]]

    return np.nan_to_num(np.array(s))


def filter_threshold(values):
    if max(values) >= intensity_treshold:
        return True
    return False



In [ ]:
df = pd.DataFrame(X_)

df['y']= y_
df['mz']= mz_
df['rt']= rt_


df = df[df[0].map(lambda  x: filter_threshold(x))]

df['y'] = df['y'].apply(lambda x: 1 if x == True else 0)

df['norm'] = df[0].apply(lambda x: NormalizeData(x) )

df['smooth'] = df['norm'].apply(lambda x: smooth(x) )
df['smooth'] = df['smooth'].apply(lambda x: x[5:-5] )

df['grad1'] = df['smooth'].apply(lambda x: np.gradient(x) )
df['grad2'] = df['grad1'].apply(lambda x: np.gradient(x) )


# Maxima & Minima
df['maxima'] = df['smooth'].apply(lambda x: scipy.signal.find_peaks(x, width=None, wlen=None, rel_height=0.5, plateau_size=None)[0] )
df['minima'] = df['smooth'].apply(lambda x: scipy.signal.find_peaks(-x, height=None, threshold=None, distance=None, prominence=None, width=None, wlen=None, rel_height=0.5, plateau_size=None)[0] )

df['maxval'] =  df[['smooth', 'maxima']].apply(lambda x: [x.smooth[i] for i in x.maxima], axis=1)
df['minval'] =  df[['smooth', 'minima']].apply(lambda x: [x.smooth[i] for i in x.minima], axis=1)

df['maxvalnorm'] =  df['maxval'].apply(lambda x: NormalizeData(x) if len(x) > 0 else [] )
df['minvalnorm'] =  df['minval'].apply(lambda x: NormalizeData(x) if len(x) > 0 else [] )

# Stats
df['maxvalstats'] =  df['maxvalnorm'].apply(lambda x: getStat(x) )
df['minvalstats'] =  df['minvalnorm'].apply(lambda x: getStat(x) )

df['maximastats'] =  df['maxima'].apply(lambda x: getStat(x) )
df['minimastats'] =  df['minima'].apply(lambda x: getStat(x) )

df['stats'] =  df['smooth'].apply(lambda x: getStat(x) )

df['fft'] = df[0].apply(lambda x: np.fft.fft(x) )
df['fftr'] = df['fft'].apply(lambda x: np.real(x) )
df['ffti'] = df['fft'].apply(lambda x: np.real(np.imag(x)) )

In [ ]:
df

In [ ]:
def ret_mats(df):
    xfft = np.hstack([np.vstack(df['fftr']), np.vstack(df['ffti']),
                      np.vstack(df['maxvalstats']),
                  np.vstack(df['minvalstats']),
                  np.vstack(df['maximastats']),
                  np.vstack(df['minimastats']),
                  np.vstack(df['stats'])
                  ])

    xstat = np.hstack([np.vstack(df['maxvalstats']),
                       np.vstack(df['minvalstats']),
                  np.vstack(df['maximastats']),
                  np.vstack(df['minimastats']),
                  np.vstack(df['stats'])
                  ])

    x = np.stack([ np.vstack(df['norm']), np.vstack(df['smooth']) ,
              np.vstack(df['grad1']) , np.vstack(df['grad2'])  ] , axis = 2)
    y = df.y.map(lambda x : float(x))

    return x,xfft,xstat, y

train, test = train_test_split(df, test_size=0.15)

mats={'train':ret_mats(train), 'test':ret_mats(test)}

In [ ]:
model = None
tf.config.run_functions_eagerly(True)

inputvec = Input(shape=(120,4))
inputfft = Input(shape=(265))
inputstat = Input(shape=(25))

lstm = LSTM(300,  return_sequences = True , return_state = False, dropout=0.2,
    recurrent_dropout=0.1, go_backwards=True)

whole_seq_output = lstm(inputvec)

lstm2 = LSTM(200, return_sequences = True , return_state = False, dropout=0.2,
    recurrent_dropout=0.1, go_backwards=True )

whole_seq_output2 = lstm2(whole_seq_output)

lstm3 = LSTM(100, return_sequences = False , return_state = True, dropout=0.2,
    recurrent_dropout=0.1, go_backwards=True )


final_memory_state, final_carry_state, whole_seq_output = lstm3(whole_seq_output2)


aux_input = tf.keras.layers.Concatenate()([final_memory_state, final_carry_state])
output1 = Dense(100,activation = 'tanh')(aux_input)
aux = Dense(1,activation = 'tanh' , name = 'AuxRnnOut' )(output1)


#dense_input = tf.keras.layers.Concatenate()([final_memory_state, final_carry_state,whole_seq_output, inputfft])

dense_input = tf.keras.layers.Concatenate()([final_memory_state, final_carry_state, inputfft,inputstat])
dense_input = tf.keras.layers.Dropout(0.15)(dense_input)

output1 = Dense(300,activation = 'tanh')(dense_input)
output1 = tf.keras.layers.Dropout(0.15)(output1)

#dense_input = tf.keras.layers.Concatenate()([output1,inputstat ])
output1 = Dense(100,activation = 'tanh')(output1)
output = tf.keras.layers.Dropout(0.1)(output1)

final = Dense(1,activation = 'tanh' , name = 'GlobalOut' )(output)


model = Model( inputs = [inputvec,inputfft,inputstat ] , outputs = [final,aux])

opt = tf.keras.optimizers.Nadam( learning_rate=0.005)
#opt = tf.keras.optimizers.RMSprop( learning_rate=0.005)

model.compile(loss='mse', optimizer=opt, metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit([mats['train'][0], mats['train'][1], mats['train'][2]], [ mats['train'][3],mats['train'][3]] , shuffle = True, epochs=400, batch_size=300 , verbose= 1)

In [ ]:
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.show()


In [ ]:
scores = model.evaluate([ mats['test'][0], mats['test'][1], mats['test'][2]], mats['test'][3], verbose=1)
print(scores)

In [ ]:
model.save('output/model_peak_' + str(int(time.time())))

In [ ]:
pre_raw = model.predict([ mats['test'][0], mats['test'][1], mats['test'][2]])

pre = [1 if  pr > 0.5 else 0 for pr in pre_raw ]


In [ ]:
fp = 0
tp = 0
fn = 0
tn = 0

cpt =0

for y_pred in pre:

    y_ref = list(mats['test'][3])[cpt]
    intensity = list(test[0])[cpt]

    if int(y_pred) != int(y_ref):

        if y_pred == 1:
            fp += 1
            plt.title( 'FP ' + str(list(test['mz'])[cpt]) + ' / '+ str(list(test['rt'])[cpt]) )
        else:
            fn += 1
            plt.title( 'FN ' + str(list(test['mz'])[cpt]) + ' / '+ str(list(test['rt'])[cpt]) )


        color = 'green' if y_pred == 1 else 'red'
        plt.plot(intensity , color = color)
        plt.show()

    else:
        if y_pred == 1:
            tp +=1
            plt.title( 'TP ' + str(list(test['mz'])[cpt]) + ' / '+ str(list(test['rt'])[cpt]) )
        else:
            tn +=1
            plt.title( 'TN ' + str(list(test['mz'])[cpt]) + ' / '+ str(list(test['rt'])[cpt]) )



        color = 'green' if y_pred == 1 else 'red'
        plt.plot(intensity , color = color)
        plt.show()


    cpt +=1



In [ ]:
total = len( mats['test'][0])
false =  tn + fp
print('Predicted Features: ', total)
print('False Predictions: ', false)
print('% Good: ', (total-false)/total*100)
print('\n')

print('TP', tp)
print('TN', tn)
print('FP', fp)
print('FN', fn)
print('\n')

print('Precision: ', (100 * tp)/ (tp+fp) )
print('Recall: ', (100 * tp/ (tp+fn) ))
print('\n')
